In [1]:
using StochasticPrograms
using LinearAlgebra
using Statistics
using HiGHS
using SparseArrays
using GLPK
using JuMP
using BenchmarkTools

In [2]:
# Construction des 2 vecteurs de tailles 3^5=243x1 pour chaque scénario possible
demands1 = [150 160 170]
demands2 = [100 120 135]
demands3 = [250 270 300]
demands4 = [300 325 350]
demands5 = [600 700 800]
probs1 = [0.25 0.5 0.25]
probs2 = [0.25 0.5 0.25]
probs3 = [0.25 0.5 0.25]
probs4 = [0.3 0.4 0.3]
probs5 = [0.3 0.4 0.3]
scenarios = collect.(Iterators.product(demands1,demands2,demands3,demands4,demands5) |> collect)
prob = prod.(Iterators.product(probs1, probs2, probs3, probs4, probs5) |> collect)

# Les constants 
factories = 3
markets = 5
trans_cost = [2.49 5.21 3.76 4.85 2.07; 1.46 2.54 1.83 1.86 4.76; 3.26 3.08 2.60 3.76 4.45]
prod_cost = 14
total_cost = trans_cost .+ prod_cost
price = 24
waste_cost = 4
cap = [500 450 650]

# J'ai choisi GLPK ici car c'est plus vite sur mon PC et le problème peut être résolu linéairement
#solver = HiGHS.Optimizer
solver = GLPK.Optimizer

GLPK.Optimizer

In [3]:
# Construction du problème de 1ere étape
function firststage(a_v,delta_v)
    m = Model(solver)

    @variable(m, Ship[i in 1:factories, j in 1:markets] >= 0)
    @variable(m, θ[i in 1:243])
    @constraint(m, capacity[i in 1:factories], sum(Ship[i,j] for j in 1:markets) <= cap[i])
    
    # On utilise la norme infinie ici pour la région de confiance
    @constraint(m, trust_region_lower[i in 1:factories, j in 1:markets], Ship[i,j]  <= delta_v + a_v[i,j])
    @constraint(m, trust_region_upper[i in 1:factories, j in 1:markets], Ship[i,j]  >= a_v[i,j] - delta_v)

    # Pas encore theta car il n'y pas encore de contrainte sur la variable
    # Ici on combine c1*Ship + c2*Prod à (c1+c2)Ship car Prod = sum(Ship)
    @objective(m, Min,
        sum(total_cost[i,j]*Ship[i,j] for i in 1:factories, j in 1:markets)
    )
    
    return m, Ship, θ, trust_region_lower, trust_region_upper
end

firststage (generic function with 1 method)

In [4]:
# Étant donné la matrice a_v et la valeur delta_v, on met à jour les contraintes de la région de confiance
function update_trust_region!(trust_region_lower,trust_region_upper,a_v, delta_v)
    for i in 1:3
        for j in 1:5
            set_normalized_rhs(trust_region_lower[i, j], delta_v + a_v[i,j])
            set_normalized_rhs(trust_region_upper[i, j], a_v[i,j] - delta_v)
        end
    end
    return nothing
end

update_trust_region! (generic function with 1 method)

In [5]:
# Problème master, c'est juste first stage + theta
# Dans le cas multi cut, le nombre de thetas = nombre de scénarios
function master_objective(m::Model, Ship, θ)   
    @objective(m, Min,
        sum(total_cost[i,j]*Ship[i,j] for i in 1:factories, j in 1:markets)
        + sum(θ[i] for i in 1:243)
    )
    return m
end

master_objective (generic function with 1 method)

In [6]:
function secondstageCore(Ship, demands)
    m = Model()

    @variable(m, Sales[j in 1:markets] >= 0)
    @variable(m, Waste[j in 1:markets] >= 0)

    # Contrainte Waste_j + Sales_j = sum(Ship[i,j]) for i -> 3 usines pour chaque j -> 3 matrice identité négative (-Tx)
    # Contrainte Sales < Demand -> pas de x -> matrice zéro  
    unit_mat = sparse([1,2,3,4,5],[1,2,3,4,5],[-1,-1,-1,-1,-1])
    T = [unit_mat unit_mat unit_mat; spzeros(5,15)]

    # Contrainte Waste_j + Sales_j = sum(Ship[i,j]) for i pas de constant -> vecteur 0
    # Contrainte Sales < Demand -> constant est la demande -> vecteur demande
    h = sparse([zeros(5); demands])


    recourseConstraints = []

    return m, Sales, Waste, recourseConstraints, h, T
end 

secondstageCore (generic function with 1 method)

In [7]:
function secondstage(Ship, demands)
    m, Sales, Waste, recourseConstraints, h, T = secondstageCore(Ship, demands)

    # Contrainte de recours
    for j = 1:markets    
        push!(recourseConstraints, @constraint(m, Waste[j] + Sales[j] == sum(Ship[i,j] for i in 1:factories)))
    end

    for j = 1:markets
        push!(recourseConstraints, @constraint(m, Sales[j] <= demands[j]))
    end

    # Q(x*,E)
    @objective(m, Min, -sum(price * Sales[j] for j in 1:markets)
                           + sum(waste_cost * Waste[j] for j in 1:markets))

    set_optimizer(m, solver)
    optimize!(m)
    
    return m, recourseConstraints, h, T
end

secondstage (generic function with 1 method)

In [8]:
# Fonction pour générer la coupe feasibility
# Notre problème n'a pas besoin de cette coupe alors c'est moins important
function secondstagefeasibility(Ship, demands)
    m, Sales, Waste, recourseConstraints, h, T = secondstageCore(Ship, demands)

    nb = 10
    @variable(m, w[1:nb] >= 0)
    for j = 1:markets    
        push!(recourseConstraints, @constraint(m, Waste[j] + Sales[j] + w[j] == sum(Ship[i,j] for i in 1:factories)))
    end

    for j = 1:markets
        push!(recourseConstraints, @constraint(m, Sales[j] - w[(j+markets)] <= demands[j]))
    end

    @objective(m, Min, sum(w[i] for i in 1:nb))

    set_optimizer(m, solver)
    optimize!(m)
    σ = dual.(recourseConstraints)  # since the objective function is a minimization, we have the correct sign.

    return σ, h, T
end

secondstagefeasibility (generic function with 1 method)

In [9]:
# Matrice Ship qui est optimal au problème d'espérance
# Point de départ idéal pour l'algo trust region
avg_result =[
0.0    0.0     0.0    0.0  500.0;
 125.0    0.0     0.0  325.0    0.0;
  35.0  118.75  272.5    0.0  200.0;
]

# Matrice Ship naive (réalisable)
a_v_naive = [1 for i in 1:factories, j in 1:markets];

In [10]:
function lshaped_multicut_trust(scenarios, prob, a_v0, verbose=false)
    nScenarios = 243
    a_v = a_v0
    
    #Choix de delta_0
    delta_v = 8
    delta_max = 100

    k = 0       # iteration index
    nfcuts = 0  # number of feasibility cuts
    nocuts = 0  # number of optimality cuts
    
    first, Ship, θ, trust_region_lower, trust_region_upper = firststage(a_v,delta_v)
    n = factories * markets
    valθ = [-Inf for i in 1:243]

    # l'indice pour compter le nombre d'itérations low-success
    stagnation = 0

    # Début de l'algo L-shaped
    while (k <= 50)
        k += 1
        if verbose
            println(first)
        end
        # Step 1
        optimize!(first)
        status = termination_status(first)
        
        if (status != MOI.OPTIMAL)
            println("Error: status ", status)
            return status, Ship, first
        end
        
        # status == MOI.OPTIMAL
        xsol = value.(Ship)
        if k!=1
            valθ = value.(θ)
        end
        
        E = zeros(n)'
        e = 0.0
        Q = 0.0 

        exact_step = false
        total_Q = 0
        total_Q_a = 0
        # Solve the second-stage programs
        # Step 2 + 3
        for iter = 1:nScenarios
            exact_step = false
            p = prob[iter]
            # On calcule m(x*) et m(a_v) pour après
            m, scstrs, h, T = secondstage(xsol, scenarios[iter])
            m_a, scstrs_a, h_a, T_a = secondstage(a_v, collect(scenarios[iter]))
            status = termination_status(m)
            
            
            if (status == MOI.INFEASIBLE)
                # Step 2, même que L-Shaped 
                # Code never enter this section
                if dual_status(m) == MOI.INFEASIBILITY_CERTIFICATE
                    # If the solver emits a infeasibility certificate, we can rely on it
                    # Note: this has been tested with HiGHS only
                    # Get dual ray from constraints (infeasibility certificate):
                    # For each constraint, try to extract its dual multiplier

                    # We cannot use all_constraints as JuMP reorganize the constraints by blocks of the same type
                    # σ = dual.(all_constraints(m; include_variable_in_set_constraints=false))
                    σ = dual.(scstrs)

                    # The sign of the dual variables depend of the sense of the optimization (min or max)
                    # Julia uses the conic duality (see https://jump.dev/MathOptInterface.jl/stable/background/duality/)
                    # As a consequence, we have to take the opposite of the dual variables in case of a minimization
                    sense = objective_sense(m)
                    if (sense == MOI.MAX_SENSE)
                        σ *= -1
                    end
                else
                    # No infeasibility certificate available
                    # We explictly solve the feasibility second-stage problem
                    σ, h, T = secondstagefeasibility(xsol, collect(scenarios[i]))
                end

                # Build a feasibility cut
                E = σ'*T
                @constraint(first, sum(E[(5*(i-1)+j)]*Ship[i,j] for i in 1:factories, j in 1:markets) >= σ'*h)
                nfcuts += 1
                break;
            elseif (status == MOI.OPTIMAL)
                # Step 3
                
                # Build the optimality cut component
                π = dual.(scstrs)
                E = p*(π'*T)
                e = p*(dot(π,h))
                
                Q = p*objective_value(m)
                Q_a = p*objective_value(m_a)

                #Aggreger pQ(x) et pQ(a_v)
                total_Q += Q
                total_Q_a += Q_a

                # Step 4 Ajouter des coupes d'optimalité
                if (nocuts == 0)
                    # add θ in the problem
                    @constraint(first, con, sum(E[(5*(i-1)+j)]*Ship[i,j] for i in 1:factories, j in 1:markets) + θ[iter] >= e)
                    master_objective(first, Ship, θ)
                else
                    check = valθ[iter]
                    if (check < Q)
                        @constraint(first, sum(E[(5*(i-1)+j)]*Ship[i,j] for i in 1:factories, j in 1:markets) + θ[iter] >= e)
                    end
                end
                nocuts += 1        
            else
                println("Error second-stage resolution: status ", status)
                return status, Ship, first
            end
        end
        # Step 5 Mise à jour a_v et delta_v + Vérifier la convergence
        prediction_reduction = sum(total_cost[i,j]*(a_v[i,j]-xsol[i,j]) for i in 1:factories, j in 1:markets)
        actual_reduction = prediction_reduction + total_Q - total_Q_a
        rho = abs(actual_reduction / prediction_reduction)

        
        if rho >= 1e-4 
            a_v = xsol
            delta_v = min(delta_v *2.0,delta_max)
        elseif stagnation <=3
            stagnation += 1
        else
            delta_v = delta_v*0.5
            stagnation = 0
        end

        update_trust_region!(trust_region_lower,trust_region_upper, a_v, delta_v)
        # Convergence
        if isapprox(rho,0.0,atol=1e-6,rtol=0) || isnan(rho)
            break
        end
    end
    optimize!(first)
    return k,nfcuts,nocuts,Ship, first, a_v
end

lshaped_multicut_trust (generic function with 2 methods)

In [11]:
k,nfcuts,nocuts,Ship, firstst, aa_v = lshaped_multicut_trust(scenarios, prob, a_v_naive)

(10, 0, 2430, VariableRef[Ship[1,1] Ship[1,2] … Ship[1,4] Ship[1,5]; Ship[2,1] Ship[2,2] … Ship[2,4] Ship[2,5]; Ship[3,1] Ship[3,2] … Ship[3,4] Ship[3,5]], A JuMP Model
Minimization problem with:
Variables: 258
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1952 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 18 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 15 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: Ship, capacity, con, trust_region_lower, trust_region_upper, θ, [0.0 0.0 … 0.0 500.0; 150.00000000000009 0.0 … 299.9999999999999 0.0; 8.452081127367983e-30 100.0 … 0.0 99.99999999999986])

In [12]:
println("Solved in ", k, " iterations.")
println("Solved using ", nfcuts, " feasibility cuts.")
println("Solved using ", nocuts, " optimality cuts.")

Solved in 10 iterations.
Solved using 0 feasibility cuts.
Solved using 2430 optimality cuts.


In [13]:
# Should be -10793 min problem > 10793 max problem
println("Objective value: ",objective_value(firstst))
println("Ship matrix: ")
display(value.(Ship))

Objective value: -10792.999999999995
Ship matrix: 


3×5 Matrix{Float64}:
   0.0    0.0    0.0    0.0  500.0
 150.0    0.0    0.0  300.0    0.0
   0.0  100.0  270.0    0.0  100.0

In [14]:
k,nfcuts,nocuts,Ship, firstst, aa_v = lshaped_multicut_trust(scenarios, prob, avg_result)

(5, 0, 1215, VariableRef[Ship[1,1] Ship[1,2] … Ship[1,4] Ship[1,5]; Ship[2,1] Ship[2,2] … Ship[2,4] Ship[2,5]; Ship[3,1] Ship[3,2] … Ship[3,4] Ship[3,5]], A JuMP Model
Minimization problem with:
Variables: 258
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 808 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 18 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 15 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: Ship, capacity, con, trust_region_lower, trust_region_upper, θ, [0.0 0.0 … 0.0 500.0; 150.00000000000003 0.0 … 300.0 0.0; 0.0 100.00000000000004 … 0.0 99.99999999999997])

In [15]:
println("Solved in ", k, " iterations.")
println("Solved using ", nfcuts, " feasibility cuts.")
println("Solved using ", nocuts, " optimality cuts.")

Solved in 5 iterations.
Solved using 0 feasibility cuts.
Solved using 1215 optimality cuts.


In [16]:
t = @btime k,nfcuts,nocuts,Ship, firstst, aa_v = lshaped_multicut_trust(scenarios, prob, a_v_naive)
t

  612.327 ms (9117298 allocations: 388.04 MiB)


(10, 0, 2430, VariableRef[Ship[1,1] Ship[1,2] … Ship[1,4] Ship[1,5]; Ship[2,1] Ship[2,2] … Ship[2,4] Ship[2,5]; Ship[3,1] Ship[3,2] … Ship[3,4] Ship[3,5]], A JuMP Model
Minimization problem with:
Variables: 258
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 1952 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 18 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 15 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: Ship, capacity, con, trust_region_lower, trust_region_upper, θ, [0.0 0.0 … 0.0 500.0; 150.00000000000009 0.0 … 299.9999999999999 0.0; 8.452081127367983e-30 100.0 … 0.0 99.99999999999986])

In [17]:
t = @btime k,nfcuts,nocuts,Ship, firstst, aa_v = lshaped_multicut_trust(scenarios, prob, avg_result)
t

  274.629 ms (4546037 allocations: 193.26 MiB)


(5, 0, 1215, VariableRef[Ship[1,1] Ship[1,2] … Ship[1,4] Ship[1,5]; Ship[2,1] Ship[2,2] … Ship[2,4] Ship[2,5]; Ship[3,1] Ship[3,2] … Ship[3,4] Ship[3,5]], A JuMP Model
Minimization problem with:
Variables: 258
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 808 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 18 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 15 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: Ship, capacity, con, trust_region_lower, trust_region_upper, θ, [0.0 0.0 … 0.0 500.0; 150.00000000000003 0.0 … 300.0 0.0; 0.0 100.00000000000004 … 0.0 99.99999999999997])